# Wheel Loader Service Request
This is a KWH HMI which allowes to request the wheel loader's *currentJobsDescription* service. Everyone can log in with their access data and authorize this HMI to call up the wheel loader's *currentJobsDescription* service. If you do not have an account yet, please contact the S³I team (s3i@kwh40.de). Of course you can also log in as one of the provided KWH user - forest expert or forest owner - if you do not want to use your own credentials.

First, all necessary modules are imported into the script, including the S³I library.

In [ ]:
from s3i import IdentityProvider, Directory, Broker, ServiceRequest, TokenType
import json
import uuid
import jwt
import time
import os
import base64
import getpass

First you have to register. You can log in as "sachverstaendiger" (forest expert) "waldbesitzer" (forest owner) or using your own account. With the specified access data of the respective user, a token is requested which authorizes this client (HMI) to call up the wheel loader service in the name of the user.

In [ ]:
print("[S3I][" + time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()) +
          "]: DEMO Wheel loader (service call), please log in as forest expert! (Sachverstaendiger)")
username = input('[S3I]: Please enter the username:')
password = getpass.getpass('[S3I]: Please enter the password:')
print("[S3I][" + time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()
                                   ) + "]: Username and password are sent to S3I IdentityProvider.")
s3i_identity_provider = IdentityProvider(grant_type='password', identity_provider_url="https://idp.s3i.vswf.dev/", realm='KWH',
                                         client_id="s3i:6f58e045-fd30-496d-b519-a0b966f1ab01", client_secret="475431fd-2c6d-4cae-bdfa-87226fff0cef", username=username, password=password)
access_token = s3i_identity_provider.get_token(TokenType.ACCESS_TOKEN)

''' decode the access token
'''
parsed_username = jwt.decode(access_token, verify=False)[
    "preferred_username"]

print("[S3I][" + time.strftime("%Y-%m-%d %H:%M:%S",
                               time.localtime()) + "]: Token received " + parsed_username + " logged in.")

The Id of this HMI and the service type for the service request are defined as global variables.

In [ ]:
 ID = "s3i:6f58e045-fd30-496d-b519-a0b966f1ab01"
 SERVICETYPE = "currentJobsDescription"

The callback function will be called and executed every time that S³I Broker delivers messages to this HMI. The result contained in the received service reply will be parsed and printed.

In [ ]:
def callback(ch, method, properties, body):
    """
    This function will be called and executed every time that S3I Broker delivers messages to the consuming application (HMI). \n
    Result contained in received service reply will be parsed and printed.

    Args:
        ch: channel used during consuming, which binds productor and consumer \n
        method: „method“ contains meta information regarding message delivery: It provides delivery flags, redelivery flags, and a routing          key that can be used to acknowledge the message. \n
        properties: properties of queue and exchange which are used during consuming \n
        body: content of message
    """

    body_str = body.decode('utf8').replace("'", '"')
    body_json = json.loads(body_str)
    print("[S3I][" + time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()) +
          "]: Service reply is received, result: " + str(body_json["results"]))
    ch.stop_consuming()

After the token has been received, it can be used to query the endpoint of the wheel loader. To query the wheel loader in the S³I Directory, you must have the rights to view the wheel loader. With the creation of your S³I account you should have got this right. In case of problems please contact the S³I team (s3i@kwh40.de).

Besides the endpoint of the wheel loder, the enpoint of the HMI is also needed to add as "ReplyToEnpoint" field in the request. The endpoint of this HMI is also queried from the directory, as it may has changed. By querying its own endpoint from the directory, the HMI is independent of changes made to its own endpoint. These changes then only need to be listed in the directory.

In [ ]:
''' authentication with JWT in S3I Directory 
'''

s3i_directory = Directory(
    s3i_dir_url="https://dir.s3i.vswf.dev/api/2/", token=access_token)

print("[S3I][" + time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()
                                   ) + "]: Authentication with Token in S3I Directory successful")
    
''' query the sender's endpoint
'''
sender_endpoint = s3i_directory.queryThingIDBased(ID+"/attributes/defaultEndpoints")[0]
    
''' query the wheel loaders's Id and endpoint
'''
wheelloaderID = s3i_directory.queryAttributeBased(
    "name", "wheelloader")[0]["thingId"]

wheelloaderServiceEndpoint = s3i_directory.queryEndpointService(
    wheelloaderID, SERVICETYPE)

receivers = [wheelloaderID]
receiver_endpoints = wheelloaderServiceEndpoint

print("[S3I][" + time.strftime("%Y-%m-%d %H:%M:%S",
                                   time.localtime()) + "]: Prepare the service request.")

parameters = dict()
msg_uuid = "s3i:" + str(uuid.uuid4())

'''
create service request
'''
servReq = ServiceRequest()
servReq.fillServiceRequest(senderUUID=ID, receiverUUID=receivers, sender_endpoint=sender_endpoint,
                           serviceType=SERVICETYPE, parameters=parameters, msgUUID=msg_uuid)

After querying the enpoints, the message is prepared. Since it is a service request, a *ServiceRequest* is instantiated and filled in.

In [ ]:
print("[S3I][" + time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()
                               ) + "]: Authentication with access token at S3I Broker")

import requests
import json
access_token = s3i_identity_provider.get_token(TokenType.ACCESS_TOKEN)
headers = {'Content-Type': 'application/json', 'Authorization': 'Bearer ' + access_token}
print(receiver_endpoints)
response = requests.post(url="https://broker.s3i.vswf.dev/"+receiver_endpoints[0] ,
                                data=json.dumps(servReq.msg), headers=headers)
print(response.text)
print(endpoints[0])


The HMI uses the token to establish a connection to the broker and sends the service request to the wheel loader's endpoint. It then waits for incoming responses. If a message arrives, the callback function is called and prints the result of the service request.

In [ ]:
    
print("[S3I][" + time.strftime("%Y-%m-%d %H:%M:%S",
                               time.localtime()) + "]: Wait for the service reply...")
demo_sachverstaendiger_app.receive(
    queue=sender_endpoint, callback=callback)

